# Chess-Bot

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment

In [2]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/wumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/wumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [3]:
train, val = cb.dataset.pos2vec_dataset()

In [4]:
pos2vec = cb.model.train_pos2vec(train, val)

Training layer 1/4
40000/40000 [==============================] - 72s 2ms/step - loss: 0.3115 - val_loss: 0.1193 - lr: 0.0050
Training layer 2/4
40000/40000 [==============================] - 75s 2ms/step - loss: 0.1077 - val_loss: 0.1033 - lr: 0.0050
Training layer 3/4
40000/40000 [==============================] - 77s 2ms/step - loss: 0.1060 - val_loss: 0.1033 - lr: 0.0050
Training layer 4/4
40000/40000 [==============================] - 79s 2ms/step - loss: 0.1057 - val_loss: 0.1037 - lr: 0.0050
Pos2Vec weights loading from /home/wumphlett/personal/Chess-Bot/models/pos2vec.h5


## DeepChess

In [5]:
train, val = cb.dataset.deepchess_dataset()

In [6]:
deepchess = cb.model.train_deepchess(train, val)

Pos2Vec weights loading from /home/wumphlett/personal/Chess-Bot/models/pos2vec.h5
20000/20000 [==============================] - 47s 2ms/step - loss: 0.5824 - accuracy: 0.6604 - val_loss: 0.4860 - val_accuracy: 0.7540 - lr: 0.0100
DeepChess weights loading from /home/wumphlett/personal/Chess-Bot/models/deepchess.h5


## Evaluation

In [7]:
deepchess = cb.model.get_deepchess()
deepchess.evaluate(train)

DeepChess weights loading from /home/wumphlett/personal/Chess-Bot/models/deepchess.h5
20000/20000 [==============================] - 42s 2ms/step - loss: 0.4810 - accuracy: 0.7566


[0.48097753524780273, 0.7566220164299011]

In [8]:
deepchess.deepchess.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dc_input_left (InputLayer)     [(None, 773)]        0           []                               
                                                                                                  
 dc_input_right (InputLayer)    [(None, 773)]        0           []                               
                                                                                                  
 pos2_vec (Pos2Vec)             (None, 100)          805100      ['dc_input_left[0][0]',          
                                                                  'dc_input_right[0][0]']         
                                                                                                  
 concatenate (Concatenate)      (None, 200)          0           ['pos2_vec[0][0]',           

## Sandbox

In [9]:
import numpy as np

train, val = cb.dataset.pos2vec_dataset()

enc = np.reshape(train[0][0][0], (1, 773))
print(pos2vec.predict(enc)[0])

1/1 [==============================] - 0s 179ms/step
[0.         0.         0.05426151 3.4136026  0.         0.
 0.49206942 3.8256633  2.2102838  2.5707335  1.6201493  5.093736
 0.33165812 0.         0.         3.1722078  2.5910048  0.83677685
 5.3608108  4.5097613  5.3332176  0.39744782 0.8848188  0.
 0.         2.0036798  0.         0.         9.573278   1.1792725
 0.         0.         0.         0.09011578 0.         0.
 0.         3.944925   0.         0.         0.         4.257313
 0.         0.         0.         0.         6.7119684  0.2539918
 0.         1.6370635  0.         0.         0.         4.038688
 0.         0.9278788  1.1670526  4.988118   0.97508526 0.
 0.6324027  2.9999447  1.3851306  0.         0.949603   1.4727182
 0.         0.         0.         0.         0.         5.5183034
 1.0147842  0.         0.         0.         2.1796873  0.50186616
 0.         2.627653   0.         7.954449   2.0838501  0.16131324
 0.         0.         2.5351393  0.8253303  0.    